In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import numpy
import datetime
import pandas as pd
import time
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import gensim
import gensim.corpora as corpora
from gensim.models import LdaMulticore
from os import path
from gensim.models import CoherenceModel
from gensim.models import ldaseqmodel
from pprint import pprint

In [ ]:
my_df = pd.read_pickle("./tweets_per_weeks.pkl")

In [ ]:
len(my_df)

# Collect tweets

In [ ]:
from_date = datetime.date(2019,3,31)
to_date = datetime.date(2020,4,1)

In [ ]:
datelist = pd.date_range(from_date, to_date, freq='2w').tolist()

In [ ]:
datelist = pd.date_range(from_date, to_date, freq='2w').tolist()
#data= dict()
#col_names =  ['weeks', 'date', 'tweet']
#my_df  = pd.DataFrame(columns = col_names)
for i in range(0,26) :
    if i in [5,10,15]:
        time.sleep(5)
    text_query = 'Insurance Canada'
    since_date = str(datelist[i].date())
    until_date = str(datelist[i+1].date())
    #count = 1000
    #Creation of query object
    tweetCriteria = manager.TweetCriteria().setQuerySearch(text_query).setSince(since_date).setUntil(until_date)#.setMaxTweets(count)
    # Creation of list that contains all tweets.
    tweets = manager.TweetManager.getTweets(tweetCriteria)
    for tweet in tweets : 
        my_df.loc[len(my_df)] = ["weeks_"+str(i+1), tweet.date.date(), tweet.text]
#      # Creating list of chosen tweet data
#     #text_tweets = [[tweet.date, tweet.text] for tweet in tweets]
#     my_df.loc[len(my_df)] = ["weeks_"+str(i+1), tweet.date, tweet.text] for tweet in tweets]

#     print("weeks_"+str(i+1))
#     #data["weeks_"+str(i+1)] = [tweet.text for tweet in tweets]

my_df.to_pickle("./tweets_per_weeks.pkl")


# preprocessing

In [ ]:
data = my_df.tweet.values.tolist()


In [ ]:
data

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [ ]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',"¯\_" ,"ツ", "_/¯",
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod ):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def preprocess_tweets(tweets ):
    # convert text to lower-case
    tweets = [tweet.lower() for tweet in tweets]
    
    # remove URLs
    tweets = [re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', tweet) for tweet in tweets]
    tweets = [re.sub(r'http\S+', '', tweet) for tweet in tweets]

    # remove emojis from tweet
    tweets = [emoji_pattern.sub(r'', tweet) for tweet in tweets]
    
    # Remove non-alphabet
    tweets = [re.sub(r'[^a-zA-Z]|(\w+:\/\/\S+)',' ', tweet) for tweet in tweets]
 
    # tokenization
    tweets_word_tokens = [word_tokenize(tweet) for tweet in tweets]
    
    # Remove short words (length < 3)
    tweets_word_tokens = [[w for w in tweet_word_tokens if len(w)>2] for tweet_word_tokens in tweets_word_tokens]
    
    # remove stopwords
    additional_stop_words=['j']
    stop_words = set(stopwords.words('english')).union(additional_stop_words)
    tweets_word_tokens_nostops = [[word for word in tweet if ( word not in stop_words and word not in emoticons and word not in string.punctuation)] for tweet in tweets_word_tokens]

       # Unigram
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(tweets_word_tokens, min_count=5, threshold=100)  # higher threshold fewer phrases.

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    # form bigram
    tweets_tokens_bigrams = make_bigrams(tweets_word_tokens_nostops, bigram_mod )
    
    return tweets_tokens_bigrams



In [ ]:

#preprocess
data_clean = preprocess_tweets(data) 

# Create Dictionary
id2word = corpora.Dictionary(data_clean)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_clean]


In [ ]:
my_df_clean=pd.read_pickle("./clean_tweets_per_weeks.pkl")

In [ ]:
data_clean = my_df_clean.tweet

# LDA

In [ ]:
dict_dur = {}
# LDA models
for i in range(24,102,2):
        start = time.time()
        lda_model = LdaMulticore(corpus,id2word=id2word, num_topics=i,passes=50, iterations =1000, workers = 3)
        end = time.time()
        dict_dur[str(i)+'topic'] = end-start
        lda_model.save(path.join('output','tweets_Lda_model_' + str(i) + "_topics"))
        del lda_model
        ps=pd.DataFrame([dict_dur])
        ps.to_pickle("./dict_dur.pkl")



In [ ]:
lda_models=[]
for i in range(2,102,2):
    lda_models.append(gensim.models.LdaModel.load("output/tweets_Lda_model_"+str(i)+"_topics"))

In [ ]:
len(lda_models)

In [ ]:

pprint(lda_models[2].print_topics())

In [ ]:
Perplexity=[]
Coherence=[]
for i in range(50):
    # Compute Perplexity
    Perplexity.append(lda_models[i].log_perplexity(corpus))  # a measure of how good the model is. lower the better.
    # Compute coherence
    Coherence.append(CoherenceModel(model=lda_models[i], texts=data_clean, dictionary=id2word, coherence='c_v').get_coherence())

In [ ]:

pd.DataFrame(Perplexity).to_pickle("./Perplexity.pkl")

pd.DataFrame(Coherence).to_pickle("./Coherence.pkl")


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Show graph
limit=102; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, Coherence)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()




In [ ]:
lda_models[17].num_topics

In [ ]:
limit=102; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, Perplexity)
plt.xlabel("Num Topics")
plt.ylabel("LogPerplexity score")
plt.legend(("Perplexity_values"), loc='best')
plt.show()

# Dynamic LDA // k = 36

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
time_slice=list(numpy.asarray(my_df['weeks'].value_counts()[my_df['weeks'].unique()]))


In [ ]:
ldaseq

In [ ]:
start_ldaseq = time.time()
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice=time_slice, num_topics=36, chunksize=1, chain_variance=0.05)
end_ldaseq = time.time()
dur_ldaseq =end_ldaseq-start_ldaseq


In [ ]:
ldaseq.save(path.join('output',"tweets_Dynamic_Lda_model_36_topics"))


In [ ]:
from gensim.models import LdaSeqModel

ldaseq = LdaSeqModel.load('./output/tweets_Dynamic_Lda_model_36_topics')

# Interpretation

In [ ]:
# extract the distribution of words of a fixed topic over time 
topic_term_matrix=dict()
for pas in range(26):
    topic_term_matrix["time"+str(pas)]= ldaseq.print_topic(17, time=pas, top_terms=34833)



In [ ]:
# extract the distribution over time of a chosen word from the topic_term_matrix
dental1=list()
# covid= list()
# dental2=list()
#cancellation=list()
for key in topic_term_matrix:
    for tupels in topic_term_matrix[key]:
        if ("dental" ) in tupels:
            dental1.append(tupels)

In [ ]:
dental1_beta=list()
for c in dental1:
    dental1_beta.append(c[1])


In [ ]:
dates= ["2019-03-31","2019-04-14","2019-04-28","2019-05-12","2019-05-26","2019-06-09","2019-06-24","2019-07-07","2019-07-21","2019-08-04","2019-08-18","2019-09-01","2019-09-15"
        ,"2019-09-29","2019-10-13","2019-10-27","2019-11-10","2019-11-24","2019-12-08","2019-12-22","2020-01-05","2020-01-19","2020-02-02","2020-02-16","2020-03-01","2020-03-15"]

df3=pd.DataFrame([dental1_beta])
# Change the column names 
df3.columns =dates
  
# Change the row indexes 
df3.index = ["dental"]

In [ ]:
df3.transpose().plot()

In [ ]:
ldaseq.print_topic_times(topic=22)[25] # evolution of topic


In [ ]:
# Document topic distribution
Doc_topic_matrix=dict()
for doc_num in range(len(my_df)):
    Doc_topic_matrix["doc"+str(doc_num)]=list(ldaseq.doc_topics(doc_num))

In [ ]:
col_names =  ['topic'+str(i) for i in range(36)]
Doc_topic_matrix = pd.DataFrame(Doc_topic_matrix,index=col_names)
Doc_topic_matrix = Doc_topic_matrix.transpose()


In [ ]:
Doc_topic_matrix

In [ ]:
Doc_topic_matrix.to_pickle("./Doc_topic_matrix.pkl")

In [ ]:
Doc_topic_matrix = pd.read_pickle("./Doc_topic_matrix.pkl")

In [ ]:
# Most representative doc of a given topic
topic_num = 17
Doc_topic_matrix[Doc_topic_matrix['topic'+str(topic_num)]==Doc_topic_matrix['topic'+str(topic_num)].max()]

In [ ]:
my_df['tweet'][174]

In [ ]:
Doc_topic_matrix[Doc_topic_matrix['topic19']==Doc_topic_matrix['topic19'].max()]

In [ ]:
[sentence  for sentence in set(list(my_df["tweet"])) if 'universal' in sentence]


In [ ]:
Doc_topic_matrix_by_time= dict()
pos=0
ind=0
for num in time_slice:
    Doc_topic_matrix_by_time["corpus"+str(ind)]=Doc_topic_matrix[pos:pos+num]
    pos= pos+num
    ind=ind+1

    

In [ ]:
# Most representative doc of a given topic by fortnight

topic_num = 3
time_stamp=19
Doc_topic_matrix_by_time["corpus"+str(time_stamp)][Doc_topic_matrix_by_time["corpus"+str(time_stamp)]['topic'+str(topic_num)]==Doc_topic_matrix_by_time["corpus"+str(time_stamp)]['topic'+str(topic_num)].max()]

In [ ]:
my_df['tweet'][28823]

In [ ]:
df= pd.DataFrame()
for key in Doc_topic_matrix_by_time:
#     print(len(pd.DataFrame(dic[key].mean())))
#     df[key]= list(pd.DataFrame(dic[key].mean()))
    df = pd.concat([df, pd.DataFrame(Doc_topic_matrix_by_time[key].mean())], axis=1)
df.columns =dates


In [ ]:
df.sum()

In [ ]:
df["max"] = df.max(axis=1)

In [ ]:
df

In [ ]:
df2=df['max']<0.04

In [ ]:
first = df[df2]
second = df[-df2]

In [ ]:
from matplotlib.ticker import PercentFormatter

figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)

first.iloc[:,:-1].transpose().plot(title="evolution of topics prorpotions ")
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=90)

